<a href="https://colab.research.google.com/github/luuun1216/Summarize_transcription/blob/main/side_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate -q
!pip install beautifulsoup4 requests -q

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import requests
from bs4 import BeautifulSoup
import json
import re
from datetime import datetime


In [ ]:
# 模型設定：使用 Hugging Face 的 Qwen 小模型
model_name = "Qwen/Qwen1.5-1.8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto")


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

In [ ]:
# 使用 text-generation pipeline
summarizer = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

def debug_selectors(soup):
    selectors = [
        (".speech__content", soup.select(".speech__content")),
        (".speech__content p", soup.select(".speech__content p")),
        (".speech-wrapper p", soup.select(".speech-wrapper p")),
    ]
    print("\n--- Selector Debug Info ---")
    for sel, result in selectors:
        print(f"Selector '{sel}': {len(result)} elements")

def extract_transcription_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    debug_selectors(soup)
    content_blocks = soup.select(".speech__content p")
    print(f"\n 共擷取到 {len(content_blocks)} 段落")
    transcript = "\n".join([t.get_text(strip=True) for t in content_blocks])
    return transcript

def clean_text(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def build_prompt(cleaned_text):
    prompt = (
        f"這是一段會議紀錄：{cleaned_text}\n\n"
        "請你根據這段內容，簡單寫出摘要與結論。\n\n"
    )
    print("\n--- Prompt Preview ---")
    print(prompt[:1000] + ("..." if len(prompt) > 1000 else ""))
    return prompt

def summarize_transcription(text):
    cleaned = clean_text(text)[:500]
    prompt = build_prompt(cleaned)
    generated = summarizer(prompt)[0]['generated_text']
    summary_part = generated.split("摘要：")[-1]
    conclusion_split = summary_part.split("結論：")
    summary = conclusion_split[0].strip()
    conclusion = conclusion_split[1].strip() if len(conclusion_split) > 1 else ""
    return {
        "summary": summary,
        "conclusion": conclusion
    }


Device set to use cuda:0


In [ ]:
# 測試用範例網址
CN_sample_url = "https://sayit.archive.tw/2025-02-02-bbc-%E6%8E%A1%E8%A8%AA"
EN_sample_url = "https://sayit.archive.tw/2025-04-03-interview-with-polly-curtis"



In [ ]:
print("Downloading transcription text...")
transcription_text = extract_transcription_text(EN_sample_url)
print("Summarizing with Qwen 1.8B...")
summary_result = summarize_transcription(transcription_text)

#  輸出結果為 JSON 格式（可讀）
print("\n--- Summary Output ---")
print(summary_result.get("summary", ""))

# print("\n--- Full Output (JSON) ---")
# print(json.dumps(summary_result, indent=2, ensure_ascii=False))

# # 寫入 JSON 檔案供下載
filename = f"summary_output_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, "w", encoding="utf-8") as f:
    json.dump(summary_result, f, ensure_ascii=False, indent=2)

# print(f"\n JSON 檔案已儲存為: {filename}")


--- Selector Debug Info ---
Selector '.speech__content': 143 elements
Selector '.speech__content p': 143 elements
Selector '.speech-wrapper p': 143 elements

✅ 共擷取到 143 段落
Summarizing with Qwen 1.8B...

--- Prompt Preview ---
這是一段會議紀錄：What are the enabling conditions for institutionalizing digital democracy? I think there essentially need to be two thresholds, one after another. The first is a widespread sense of urgency about something the British call a “wicked problem”—like there’s not a single actor that can solve it in a Pareto improvement kind of way, but rather, it requires everybody to sense-make together. The trade deal with Beijing in 2014 was certainly of this shape. The first generation of AI manipulation algorith

請你根據這段內容，簡單寫出摘要與結論。



--- Summary Output ---
在組織化數字民主的實現條件中，有一些關鍵性的條件需要滿足。這些條件主要包括兩個階段，一個是在全球對某一問題具有急迫感（如北京貿易談判中的「壞問題」），另一個是每個人都需要意識到這種問題的必要性並進行共同創造解決方案的努力。首先，解決這個問題的手段應該是通過協調和共識來實現，而不是通過單點的改革或技術進步。其次，這種解決方案必須符合“帕累托優化”的原則，即不是每個人都可以從改革或技術改善中受益，而是需要所有人共同努力才能實現真正的